In [16]:
import numpy as np
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
from tqdm import tqdm
from numpy import nan
import math
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
from transformers import BertTokenizer, TFBertModel
from tensorflow_addons.text import crf_log_likelihood, crf_decode
from tensorflow.keras.layers import LSTM, Dense, Bidirectional
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy

import transformers
from transformers import BertTokenizerFast
from transformers import TFBertModel

from sklearn import preprocessing
from sklearn.model_selection import train_test_split

c:\Users\u1158286\AppData\Local\Programs\Python\Python311\Lib\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [17]:
class NERModel(tf.keras.Model):
   def __init__(self, bert_model, num_labels):
       super(NERModel, self).__init__()
       self.bert = bert_model
       self.bilstm = Bidirectional(LSTM(128, return_sequences=True))
       self.classifier = Dense(num_labels)
       self.transition_params = tf.Variable(tf.random.uniform(shape=(num_labels, num_labels)))
       
   def call(self, inputs, training=False):
       input_ids, attention_mask = inputs
       bert_output = self.bert(input_ids, attention_mask=attention_mask)[0]
       lstm_output = self.bilstm(bert_output)
       logits = self.classifier(lstm_output)
       return logits
   
   def compute_loss(self, logits, labels, seq_lengths):
       log_likelihood, self.transition_params = crf_log_likelihood(
           logits, labels, seq_lengths, self.transition_params
       )
       return -tf.reduce_mean(log_likelihood)

In [22]:
# bert_model = TFBertModel.from_pretrained(r'C:\Users\u1158286\OneDrive - IQVIA\Desktop\NER\bert_model\bert-base-uncased',from_pt = True)
# ner_model = NERModel(bert_model,16)
# optimizer = Adam(learning_rate=3e-5)